In [1]:
import os
import pandas as pd
import cx_Oracle
import pymysql
from ftplib import FTP
import requests
import json
import psycopg2

HEADERS =  {
    'Authorization':""
}

REFRESH_TOKEN = ''

domain = 'https://dashboard-rmbmss.titan.com/'

def get_data(sql,col_lst):
    #连接数据库
    conn=cx_Oracle.connect('ONL1/Rmb2_2015@200.80.24.87:1521/RMBDB',encoding = 'UTF-8',nencoding = 'UTF-8')
    curs=conn.cursor()
    #获取当日实时成交数据
    curs.execute(sql)
    rows=curs.fetchall()
    #转为dataframe格式
    df=pd.DataFrame(rows)
    if not df.empty:
        df.columns = col_lst
    return df

def get_data_sys(sql,col_lst):
    #连接数据库
    conn=cx_Oracle.connect('SYS/Rmb2_2015@200.80.24.87:1521/RMBDB',encoding = 'UTF-8',nencoding = 'UTF-8',mode = cx_Oracle.SYSDBA)
    curs=conn.cursor()
    #获取当日实时成交数据
    curs.execute(sql)
    rows=curs.fetchall()
    #转为dataframe格式
    df=pd.DataFrame(rows)
    if not df.empty:
        df.columns = col_lst
    return df

def get_mysql_data(sql):
    #连接数据库
    conn=pymysql.connect(host = '127.0.0.1',user = 'root',passwd='root',port = 3306,db='cfets',charset = 'utf8')
    curs=conn.cursor()
    #获取当日实时成交数据
    curs.execute(sql)
    rows=curs.fetchall()
    #转为dataframe格式
    lst = []   
    for i in rows:
        lst.append(list(i))
    return lst

#保留两位小数
def num2(x):
    x = float(x)
    return round(x,2)

#四舍五入
def int2(x):
    x = float(x)
    return int(x+0.5)

#判断境外机构
def get_outland(x):
    if '境外' in x or x == '主权财富基金' or x == '国际金融组织':
        return 1
    else:
        return 0

def upload_ftp(file,path):
    bufsize=1024
    ftp=FTP()
    ftp.connect("200.31.138.9",21)
    ftp.login()
    ftp.encoding='gbk'
    ftp.cwd(path)
    f=open(file,'rb')
    ftp.storbinary("STOR %s"%os.path.basename(file),f,bufsize)
    ftp.set_debuglevel(0)
    f.close()
    ftp.quit()  


def get_loginapi():
    global HEADERS
    global REFRESH_TOKEN
    url = domain + 'loginapi'
    data = {'username':'admin',
            'password':'E5be33Cb967bHf7e'}
    res = requests.get(url,data=data,verify = False)
    js = res.text
    # print('----------------------------------------',js)
    token_js = json.loads(js)
    token = token_js.get('access_token','')
    REFRESH_TOKEN = token_js.get('refresh_token','')
    HEADERS['Authorization'] = 'Bearer ' + token
    # print('accessssssssssssssssssss',HEADERS)
    print('首次登陆，获取token成功')

def get_refreshapi():
    global HEADERS
    global REFRESH_TOKEN

    url = os.path.join(domain , 'refreshapi')
    data = {'rf_token':REFRESH_TOKEN}
    res = requests.get(url,data=data,verify = False)
    js = res.text
    # print('----------------------------------------',js)
    token_js = json.loads(js)
    token = token_js.get('access_token','')
    REFRESH_TOKEN = token_js.get('refresh_token','')
    HEADERS['Authorization'] = 'Bearer ' + token
    # print('refreshhhhhhhhhhhhhhhhhhh',HEADERS)
    print('刷新token成功！！')


def gp_sql(sql):
    url = os.path.join(domain , 'gp_sql')
    data = {'sql':sql}
    res = requests.post(url,data=data,headers=HEADERS,verify = False)
    return res.text

def gp_params(fields,tables,ons,wheres,groups,orders):
    url = os.path.join(domain , 'gp_params')
    data = {'fields':fields,
            'tables':tables,
            'ons':ons,
            'wheres':wheres,
            'groups':groups,
            'orders':orders}

    res = requests.post(url,data=data,headers = HEADERS,verify = False)
    return res.text


def gp_data(js):
    res_js = json.loads(js)
    # print('-----------------------------',res_js)
    data = res_js.get('data')
    df= pd.DataFrame(data)
    col_lst = df.columns

    for key in col_lst:
        try:
            df[key] = df[key].astype(float)
        except:
            continue
    return df


def gp_data2(js):
    res_js = json.loads(js)
    # print('-----------------------------',res_js)
    data = res_js.get('data')
    df= pd.DataFrame(data)
    return df


def pg_select(sql):
    conn = psycopg2.connect(database='sdpdb', user='sdpuser', password='123456', host='200.31.175.88', port='5432')
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    cols = [desc[0] for desc in cur.description]
    df = pd.DataFrame(rows,columns=cols)
    conn.commit()
    conn.close()
    return df

In [2]:
# transaction
# Display all the columns 
sql = """
select *
from  dpa.cbt_dl_dtl
where txn_dt = '2023-07-27'
limit 30
"""
df_trade = pg_select(sql)
pd.set_option("display.max_columns", None)
print(df_trade.head())

               dl_cd dt_cnfrm     txn_dt                     dl_tm  \
0  CBT20230727211342     None 2023-07-27 2023-07-27 16:13:00+08:00   
1  CBT20230727211337     None 2023-07-27 2023-07-27 16:12:57+08:00   
2  CBT20230727211328     None 2023-07-27 2023-07-27 16:12:44+08:00   
3  CBT20230727211290     None 2023-07-27 2023-07-27 16:12:12+08:00   
4  CBT20230727311731     None 2023-07-27 2023-07-27 16:11:58+08:00   

                    bsns_tm          qt_rqst_cd trd_rcrd_cd prdct_cd  \
0 2023-07-27 16:13:00+08:00  230727090411007492        None      CBT   
1 2023-07-27 16:12:57+08:00  230727090412004893        None      CBT   
2 2023-07-27 16:12:44+08:00  230727090412006208        None      CBT   
3 2023-07-27 16:12:12+08:00  230727090412007217        None      CBT   
4 2023-07-27 16:11:58+08:00                None        None      CBT   

  trdng_md_cd trdng_mthd_cd qt_md qt_tp chnl_tp   chnl prmrkt_bond_f qt_src  \
0         QDM           RFQ  None  None       4  CFETS             

In [3]:
import pandas as pd

# query the transaction data
sql = """
select *
from  dpa.cbt_dl_dtl
where txn_dt = '2023-07-05' and trdng_md_cd = 'NDM'
limit 30000
"""
df_trade = pg_select(sql)

columns_to_export = [
    'dl_cd', 'trdng_md_cd', 'trdng_mthd_cd', 'bond_cd', 'bnds_nm', 'net_prc', 'yld_to_mrty', 'nmnl_vol', 'amnt', 'byr_instn_cn_full_nm', 'slr_instn_cn_full_nm'
]

# extract the columns we use
ndm_transaction_list = df_trade#[columns_to_export]

print(ndm_transaction_list.shape)
print(ndm_transaction_list.head())

# save the transaction data
ndm_transaction_list.to_csv("ndm_transaction_list_all_20230705.csv")

(12396, 146)
               dl_cd dt_cnfrm     txn_dt                     dl_tm  \
0  CBT20230705312184     None 2023-07-05 2023-07-05 17:09:12+08:00   
1  CBT20230705312312     None 2023-07-05 2023-07-05 17:43:48+08:00   
2  CBT20230705312197     None 2023-07-05 2023-07-05 17:10:54+08:00   
3  CBT20230705312274     None 2023-07-05 2023-07-05 17:25:23+08:00   
4  CBT20230705312263     None 2023-07-05 2023-07-05 17:21:49+08:00   

                    bsns_tm qt_rqst_cd trd_rcrd_cd prdct_cd trdng_md_cd  \
0 2023-07-05 17:09:12+08:00       None        None      CBT         NDM   
1 2023-07-05 17:43:48+08:00       None        None      CBT         NDM   
2 2023-07-05 17:10:54+08:00       None        None      CBT         NDM   
3 2023-07-05 17:25:23+08:00       None        None      CBT         NDM   
4 2023-07-05 17:21:49+08:00       None        None      CBT         NDM   

  trdng_mthd_cd qt_md qt_tp chnl_tp   chnl prmrkt_bond_f qt_src dl_st  \
0     Negotiate  None  None       4  CFETS

In [90]:
# query the valuation market data
sql = """
select *
from  dpa.cntrl_dpstry_vltn
where vltn_dt = '2023-07-05'
limit 30000
"""
df_valuation = pg_select(sql)

columns_to_export = [
    'vltn_dt', 'bond_cd', 'bnds_nm', 'bond_tp', 'exrcs_f', 'vltn_full_prc', 'vltn_net_prc', 'yld_to_mrty', 'rcmnd_prc_indctr'
]

# extract the columns we use
valuation_list = df_valuation[columns_to_export]

print(valuation_list.shape)
print(valuation_list.head())

# save the valuation data
valuation_list.to_csv("valuation_list_20230705.csv")

(30000, 22)
     vltn_dt    bond_cd      bnds_nm bond_tp       clsng_vltn_tm exrcs_f  \
0 2023-07-05  112205140  22建设银行CD140      26 2023-07-05 19:12:24       0   
1 2023-07-05  112217147  22光大银行CD147      26 2023-07-05 19:12:24       0   
2 2023-07-05  112313049  23浙商银行CD049      26 2023-07-05 19:12:24       0   
3 2023-07-05  112313041  23浙商银行CD041      26 2023-07-05 19:12:24       0   
4 2023-07-05  112313012  23浙商银行CD012      26 2023-07-05 19:12:23       0   

   vltn_full_prc   vltn_net_prc   ttm ttm_yrly   yld_to_mrty exrcs_yld cpn_tp  \
0  99.7255000000  98.0883000000  None      51D  1.9701000000      None      0   
1  99.8506000000  97.9959000000  None      28D  1.9500000000      None      0   
2  99.2313000000  98.2527000000  None     134D  2.1100000000      None      0   
3  99.2712000000  98.2395000000  None     127D  2.1099000000      None      0   
4  98.8226000000  97.6424000000  None     195D  2.2301000000      None      0   

  mdfd_drtn   vltn_cnvxty   exchng_rate bond

In [4]:
# local load the price market data
df_mkt_price = pd.read_csv("CBT_DL_MKT_INFO20230705.csv")

columns_to_export = [
    'mkt_data_upd_tm', 'bond_cd', 'opng_net_prc', 'hghst_net_prc', 'lwst_net_prc', 'clsng_net_prc', 'wghtd_avg_net_prc', 'trd_vol'
]

# extract the columns we use
mkt_price_list = df_mkt_price#[columns_to_export]

print(mkt_price_list.shape)
print(mkt_price_list.head())

# save the valuation data
mkt_price_list.to_csv("mkt_price_list_all_20230705.csv")

(33060, 58)
                    mkt_data_upd_tm  dt_cnfrm  cfets_unfd_cd prdct_cd  \
0  2023-07-05 09:03:29.127000+08:00       NaN            NaN      CBT   
1  2023-07-05 09:11:15.774000+08:00       NaN            NaN      CBT   
2  2023-07-05 09:18:05.739000+08:00       NaN            NaN      CBT   
3  2023-07-05 09:20:02.102000+08:00       NaN            NaN      CBT   
4  2023-07-05 09:21:56.655000+08:00       NaN            NaN      CBT   

  trdng_md_cd trdng_mthd_cd  mkt_scp  chnl_tp   chnl  mkt_data_sub_tp  \
0         ALL           ALL        1        4  CFETS                4   
1         ALL           ALL        1        4  CFETS                4   
2         ALL           ALL        1        4  CFETS                4   
3         ALL           ALL        1        4  CFETS                4   
4         ALL           ALL        1        4  CFETS                4   

   mkt_qtn_st  bond_cd  bnds_nm  bond_indctr_list  prmrkt_bond_f  cash_bond_f  \
0         NaN   230012      N